In [ ]:
import requests

API_URL = "https://bills-api.parliament.uk/api/v1/Bills"

def get_bills():
    bills = []
    for i in range(0, 75):
        # print(i)
        response = requests.get(f"{API_URL}?skip={i*10}")
        print(len(bills))
        if response.status_code != 200:
            break
            
        bills.extend(response.json()["items"])
    # print(bills)
    # print(len(bills))
    return bills

bills = get_bills()

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.8 MB/s eta 0:00:00


In [ ]:
import transformers
import pandas as pd

tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased")
model = transformers.AutoModel.from_pretrained("bert-base-uncased")

def get_bert_embeddings(text):
    tokens = tokenizer(text, return_tensors="pt", padding=True)
    outputs = model(**tokens)
    return outputs.pooler_output

df = pd.DataFrame(bills)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=3e62485aa9e7b9bbf1cf21b3d55d8cb747e88069c44ea877263a490d3692454c
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

df = pd.DataFrame(bills)

df = df.drop_duplicates(subset=["shortTitle"])

# df = df.head(250)

df["shortTitleLower"] = df["shortTitle"].str.lower()
df["shortTitleLower"] = df["shortTitleLower"].str.replace(r"[^a-zA-Z0-9 ]", "")

df["shortTitleLower"] = df["shortTitleLower"].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

# calculate embeddings
df["embedding"] = df["shortTitleLower"].apply(lambda x: model.encode(x))

print(len(df))

<ipython-input-127-785bed6e3b9c>:12: FutureWarning:

The default value of regex will change from True to False in a future version.



609


In [ ]:
import nltk

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# bar chart of word counts
from collections import Counter
import plotly.express as px

# remove stopwords
from nltk.corpus import stopwords

stop = stopwords.words("english")

all_words = df["shortTitleLower"].str.split(expand=True).stack().value_counts()

all_words = all_words.drop("bill")
all_words = all_words.drop("hl")
all_words = all_words.drop("act")

fig = px.bar(all_words[:20])
fig.show()

In [ ]:
from sklearn.manifold import TSNE
import numpy as np

tsne = TSNE(n_components=2)

print(len(df))

df1 = df[df["currentHouse"] == "Commons"]

print(len(df1))

X = np.array(df1["embedding"].tolist()).squeeze()

tsne_results = tsne.fit_transform(X)

df1["tsne-2d-one"] = tsne_results[:, 0]
df1["tsne-2d-two"] = tsne_results[:, 1]
df1["currentStageDescription"] = df1["currentStage"].apply(lambda x: x["description"])
# undefeated bills that are not drafts
df1 = df1[df1["isDefeated"] == False]
df1 = df1[df1["billTypeId"] != 9]

# df1["color"] = "red"
# df1.loc[df1["isAct"] == True, "color"] = "blue"

fig = px.scatter(
    df1,
    x="tsne-2d-one",
    y="tsne-2d-two",
    size_max=18,
    title="t-SNE applied to BERT embeddings of Bill titles",
    hover_data=["shortTitle"],
    color="currentStageDescription"
)
fig.show()

609
391


<ipython-input-148-d391c1e87042>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-148-d391c1e87042>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-148-d391c1e87042>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=0).fit(df["embedding"].tolist())

df["cluster"] = kmeans.labels_

X = np.array(df["embedding"].tolist()).squeeze()

# show cluster
fig = px.scatter(
    df,
    x=X[:, 0],
    y=X[:, 1],
    color="cluster",
    hover_name="shortTitle",
)
fig.show()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

